# 🗑️ YOLOv8 Waste Detection Training
## Kampus 1 UNJANI Yogyakarta - Object Detection

**Dataset**: Roboflow garbage_best (3485 train, 487 test images)

**Model**: YOLOv8 Nano (lightweight untuk deployment)

**Output**: Bounding boxes untuk deteksi lokasi sampah

## 📦 Step 1: Install Dependencies

In [ ]:
# Install Ultralytics YOLOv8
!pip install ultralytics roboflow -q

# Import libraries
from ultralytics import YOLO
from roboflow import Roboflow
import os
from pathlib import Path
from google.colab import drive

print("✅ Dependencies installed!")

## 💾 Step 2: Mount Google Drive (Optional - untuk save model)

In [ ]:
# Mount Google Drive untuk save model
drive.mount('/content/drive')

# Create output directory
output_dir = Path('/content/drive/MyDrive/waste-detection-yolo')
output_dir.mkdir(exist_ok=True)

print(f"✅ Output directory: {output_dir}")

## 📥 Step 3: Download Dataset dari Roboflow

**IMPORTANT**: Ganti `YOUR_API_KEY` dengan API key dari Roboflow!

Cara dapat API key:
1. Login ke Roboflow
2. Klik profile → Settings → API Key
3. Copy paste ke cell di bawah

In [ ]:
# Download dataset dari Roboflow
# ⚠️ GANTI 'YOUR_API_KEY' dengan API key kamu dari Roboflow!

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="YOUR_API_KEY")
project = rf.workspace("smart-india-hackathon-2023").project("garbage_best")
version = project.version(1)
dataset = version.download("yolov8")

print(f"✅ Dataset downloaded to: {dataset.location}")

## 🔍 Step 4: Inspect Dataset

In [ ]:
# Check dataset structure
import yaml

data_yaml = Path(dataset.location) / 'data.yaml'

with open(data_yaml, 'r') as f:
    data_config = yaml.safe_load(f)

print("📊 Dataset Configuration:")
print(f"  Train images: {data_config.get('train', 'N/A')}")
print(f"  Val images: {data_config.get('val', 'N/A')}")
print(f"  Test images: {data_config.get('test', 'N/A')}")
print(f"  Classes: {data_config.get('names', [])}")
print(f"  Number of classes: {data_config.get('nc', 0)}")

## 🚀 Step 5: Train YOLOv8 Model

**Model Options**:
- `yolov8n.pt` - Nano (fastest, smallest) ✅ RECOMMENDED
- `yolov8s.pt` - Small
- `yolov8m.pt` - Medium
- `yolov8l.pt` - Large
- `yolov8x.pt` - Extra Large

In [ ]:
# Initialize YOLOv8 Nano model
model = YOLO('yolov8n.pt')

# Training parameters
results = model.train(
    data=str(data_yaml),
    epochs=100,              # Jumlah epoch (bisa dikurangi jadi 50 untuk cepat)
    imgsz=640,               # Image size
    batch=16,                # Batch size (kurangi jika OOM)
    name='waste_detection',  # Experiment name
    patience=20,             # Early stopping patience
    save=True,               # Save checkpoints
    device=0,                # GPU device (0 = first GPU)
    workers=2,               # Number of workers
    project='runs/detect',   # Project directory
    exist_ok=True,           # Overwrite existing
    pretrained=True,         # Use pretrained weights
    optimizer='auto',        # Optimizer
    verbose=True,            # Verbose output
    seed=42,                 # Random seed
    deterministic=True,      # Deterministic mode
    single_cls=False,        # Multi-class detection
    rect=False,              # Rectangular training
    cos_lr=False,            # Cosine LR scheduler
    close_mosaic=10,         # Disable mosaic augmentation for final epochs
    resume=False,            # Resume training
    amp=True,                # Automatic Mixed Precision
    fraction=1.0,            # Dataset fraction to train on
    profile=False,           # Profile ONNX and TensorRT speeds
    freeze=None,             # Freeze layers
    lr0=0.01,                # Initial learning rate
    lrf=0.01,                # Final learning rate
    momentum=0.937,          # SGD momentum
    weight_decay=0.0005,     # Optimizer weight decay
    warmup_epochs=3.0,       # Warmup epochs
    warmup_momentum=0.8,     # Warmup momentum
    warmup_bias_lr=0.1,      # Warmup bias learning rate
    box=7.5,                 # Box loss gain
    cls=0.5,                 # Class loss gain
    dfl=1.5,                 # DFL loss gain
    pose=12.0,               # Pose loss gain
    kobj=1.0,                # Keypoint obj loss gain
    label_smoothing=0.0,     # Label smoothing
    nbs=64,                  # Nominal batch size
    hsv_h=0.015,             # HSV-Hue augmentation
    hsv_s=0.7,               # HSV-Saturation augmentation
    hsv_v=0.4,               # HSV-Value augmentation
    degrees=0.0,             # Rotation augmentation
    translate=0.1,           # Translation augmentation
    scale=0.5,               # Scale augmentation
    shear=0.0,               # Shear augmentation
    perspective=0.0,         # Perspective augmentation
    flipud=0.0,              # Flip up-down augmentation
    fliplr=0.5,              # Flip left-right augmentation
    mosaic=1.0,              # Mosaic augmentation
    mixup=0.0,               # Mixup augmentation
    copy_paste=0.0,          # Copy-paste augmentation
)

print("\n✅ Training completed!")

## 📊 Step 6: Evaluate Model

In [ ]:
# Validate model on test set
metrics = model.val()

print("\n📊 Model Performance:")
print(f"  mAP50: {metrics.box.map50:.3f}")
print(f"  mAP50-95: {metrics.box.map:.3f}")
print(f"  Precision: {metrics.box.mp:.3f}")
print(f"  Recall: {metrics.box.mr:.3f}")

## 🧪 Step 7: Test Inference

In [ ]:
# Test on sample images
test_images = list(Path(dataset.location).glob('test/images/*.jpg'))[:5]

for img_path in test_images:
    results = model.predict(str(img_path), conf=0.25)
    
    # Display results
    for r in results:
        print(f"\n📸 Image: {img_path.name}")
        print(f"  Detected {len(r.boxes)} objects")
        
        # Show image with boxes
        import matplotlib.pyplot as plt
        from PIL import Image
        
        img_with_boxes = r.plot()
        plt.figure(figsize=(12, 8))
        plt.imshow(img_with_boxes)
        plt.axis('off')
        plt.title(f"{img_path.name} - {len(r.boxes)} objects detected")
        plt.show()

## 💾 Step 8: Export Model

In [ ]:
# Get best model path
best_model_path = Path('runs/detect/waste_detection/weights/best.pt')

# Copy to Google Drive
import shutil

drive_model_path = output_dir / 'waste_yolo_best.pt'
shutil.copy(best_model_path, drive_model_path)

print(f"✅ Model saved to: {drive_model_path}")
print(f"\n📦 Model size: {drive_model_path.stat().st_size / 1024 / 1024:.2f} MB")

# Also save to Colab for download
colab_model_path = Path('/content/waste_yolo_best.pt')
shutil.copy(best_model_path, colab_model_path)

print(f"\n📥 Download model from: {colab_model_path}")
print("   (Klik kanan di file browser → Download)")

## 📈 Step 9: View Training Results

In [ ]:
# Display training curves
from IPython.display import Image, display

results_dir = Path('runs/detect/waste_detection')

print("📊 Training Results:")

# Results plot
if (results_dir / 'results.png').exists():
    display(Image(filename=str(results_dir / 'results.png')))

# Confusion matrix
if (results_dir / 'confusion_matrix.png').exists():
    print("\n🎯 Confusion Matrix:")
    display(Image(filename=str(results_dir / 'confusion_matrix.png')))

# PR curve
if (results_dir / 'PR_curve.png').exists():
    print("\n📈 Precision-Recall Curve:")
    display(Image(filename=str(results_dir / 'PR_curve.png')))

## 🎯 Step 10: Model Info & Next Steps

In [ ]:
print("="*70)
print("🎉 YOLOv8 Training Completed!")
print("="*70)
print("\n📦 Model Files:")
print(f"  Best model: {best_model_path}")
print(f"  Google Drive: {drive_model_path}")
print(f"  Download: {colab_model_path}")
print("\n📊 Performance:")
print(f"  mAP50: {metrics.box.map50:.3f}")
print(f"  mAP50-95: {metrics.box.map:.3f}")
print("\n🚀 Next Steps:")
print("  1. Download model: waste_yolo_best.pt")
print("  2. Copy to project: models/waste_yolo_best.pt")
print("  3. Update app.py untuk integrasi YOLO")
print("  4. Test inference di localhost")
print("  5. Deploy ke server")
print("="*70)